In [ ]:
import vertexai
import os
import pandas as pd
from langchain import SQLDatabase
from langchain import PromptTemplate
from langchain_google_vertexai import VertexAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import OutputFixingParser


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ""
vertexai.init(project="", location="europe-west3")
sqlalchemy_url = "bigquery://"

In [ ]:
TABLE_SCHEMA = """CREATE TABLE `project_name.dataset_name.table_name`
    (
    billing_account_id STRING,
    service STRUCT<id STRING, description STRING>,
    sku STRUCT<id STRING, description STRING>,
    usage_start_time TIMESTAMP,
    usage_end_time TIMESTAMP,
    project STRUCT<id STRING, number STRING, name STRING, labels ARRAY<STRUCT<key STRING, value STRING>>, ancestry_numbers STRING, ancestors ARRAY<STRUCT<resource_name STRING, display_name STRING>>>,
    labels ARRAY<STRUCT<key STRING, value STRING>>,
    system_labels ARRAY<STRUCT<key STRING, value STRING>>,
    location STRUCT<location STRING, country STRING, region STRING, zone STRING>,
    tags ARRAY<STRUCT<key STRING, value STRING, inherited BOOL, namespace STRING>>,
    export_time TIMESTAMP,
    cost FLOAT64,
    currency STRING,
    currency_conversion_rate FLOAT64,
    usage STRUCT<amount FLOAT64, unit STRING, amount_in_pricing_units FLOAT64, pricing_unit STRING>,
    credits ARRAY<STRUCT<name STRING, amount FLOAT64, full_name STRING, id STRING, type STRING>>,
    invoice STRUCT<month STRING>,
    cost_type STRING,
    adjustment_info STRUCT<id STRING, description STRING, mode STRING, type STRING>,
    cost_at_list FLOAT64
    )
    PARTITION BY DATE(_PARTITIONDATE);"""

In [ ]:
llm = VertexAI(model_name="code-bison@002", temperature=0.4)
db = SQLDatabase.from_uri(sqlalchemy_url,
                          include_tables=[""],
                          custom_table_info={ "" : TABLE_SCHEMA},
                          view_support=True,
                          engine_args={'location': 'EU'})

In [ ]:
_DEFAULT_TEMPLATE = """
You are a {dialect} expert. Given an input question, creat a syntactically correct {dialect} SQL query to run.
Please ensure that given query can be executed on {dialect}.
Note: Do not return ```sql ``` enclosed results.

Use the following format:

SQLQuery: "SQL Query to run"

Expalantion: "Explain your query in a few words." 

Use the following tables:
{table_info}

Question: {input}

"""

In [ ]:
from typing import Optional


class SQLCode(BaseModel):
    code: str = Field(description="SQL code to run")
    explanation: Optional[str] = Field(description="Explanation of the SQL code")

parser = PydanticOutputParser(pydantic_object=SQLCode)

prompt = PromptTemplate(
    template=_DEFAULT_TEMPLATE,
    input_variables=["input", "table_info", "dialect"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


output_fixining_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
chain = prompt | llm | output_fixining_parser 

In [ ]:
q1 = "Give me last day total gcp cost."
q2 = "Give me last day total gcp cost for project 'xyz'."
q3 = "I need total slot usage for last day the SKU description is 'Analysis Slots Attribution'."
q4 = "I need total cost for last month for each sku description"
q5 = "Can you provide cost distribution for each project by days in the columns?"

In [ ]:
question: str = q5

input = dict(input=question, 
            table_info=db.get_table_info(), 
            dialect=db.dialect)
query = chain.invoke(input)
result = db.run(query.code)

print(f'Query: {query.code}')
print(f'Explanation: {query.explanation}')
eval(result)

In [ ]:
pd.DataFrame(eval(result))